# Capstone Project - The Battle of Neighborhoods(Part 1)

## 1. Description of the Problem

__London__ is the __capital__ and __largest city__ of __England__ and the __United Kingdom__. London has been a major settlement for two millennia. London is considered to be one of the world's most important __global__ cities and has been called the world's most powerful, most desirable, most influential, most visited, most expensive, sustainable, most investment-friendly,and most-popular-for-work city. London also has largest __financial__ institutions and is also home to __top-ranked__ universities in the world. ([Wiki](https://en.wikipedia.org/wiki/London))

London has always been a __dream__ city for me personally to move and live there. To move to such a vibrant city, one of the main concerns is to find good __affordable__ accomodation in a __safe__ neighborhood. Therfore in this project, I intend to explore London neighborhoods and find the best the neighborhood to move in. This project could potentially help other people who have similar interests to move to London and live there.  

Through this project, I would like to get to know the safest borough (less crime rate as one of main factor) and its neighborhoods venues.

## 2. Data description

To get the desired results, following data sources will be used. 
1. London Recorded Crime ([Crime Data](https://data.london.gov.uk/dataset/recorded_crime_summary))(Last 24 months data is considered for evaluation)
2. List of London Boroughs ([List](https://en.wikipedia.org/wiki/List_of_London_boroughs))
3. Foursquare APIs are used too explore neighborhoods and target different venues according to the presence of needed facilities. The data is stored in the dataframe and used for visualization.






In [ ]:
!conda install -c anaconda beautifulsoup4 --yes

In [ ]:
from bs4 import BeautifulSoup
import datetime
import re
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import csv
import xml
import json # library to handle JSON files
import requests # library to handle requests

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Foursquare APIs
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

print("Libraries Import Done!")

### 2.1 Getting London Crime Data

Reading the data from the Data source and storing in Dataframe

In [39]:
#Direct Link to File
#https://data.london.gov.uk/download/recorded_crime_summary/d2e9ccfc-a054-41e3-89fb-53c2bc3ed87a/MPS%20Borough%20Level%20Crime%20%28most%20recent%2024%20months%29.csv
#file_loc= https://data.london.gov.uk/download/recorded_crime_summary/d2e9ccfc-a054-41e3-89fb-53c2bc3ed87a/MPS%20Borough%20Level%20Crime%20%28most%20recent%2024%20months%29.csv

# Read crime records data
crime_lon = pd.read_csv("MPS Borough Level Crime (most recent 24 months).csv")
#print("Data successfully read")
crime_lon.head()

,MajorText,MinorText,LookUp_BoroughName,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005
0,Arson and Criminal Damage,Arson,Barking and Dagenham,12,6,5,3,8,5,1,5,2,5,5,11,3,5,3,6,9,8,6,4,5,6,2,2
1,Arson and Criminal Damage,Criminal Damage,Barking and Dagenham,123,127,101,107,132,105,88,97,127,138,130,140,113,134,118,109,109,97,121,97,103,107,80,88
2,Burglary,Burglary - Business and Community,Barking and Dagenham,33,30,18,33,32,39,33,45,24,29,27,21,27,31,35,37,30,30,25,31,17,28,29,16
3,Burglary,Burglary - Residential,Barking and Dagenham,77,94,84,99,94,106,164,114,107,99,96,114,96,71,67,80,97,114,130,116,123,97,57,41
4,Drug Offences,Drug Trafficking,Barking and Dagenham,6,8,7,10,7,7,4,5,2,6,5,9,6,11,7,7,10,12,3,13,4,6,11,11


#### Crime Data Exploration of London Boroughs

Identifying the Crime rates in each of Boroughs

In [40]:
# Dimension of Data
crime_lon.shape

(1566, 27)

In [41]:
# Names of Boroughs in London City
crime_lon['LookUp_BoroughName'].unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
       'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
       'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Lambeth', 'Lewisham',
       'London Heathrow and London City Airports', 'Merton', 'Newham',
       'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
       'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster'],
      dtype=object)

In [42]:
# Get the total count of incidents and show them in a column(Sum)
crime_lon['Sum'] = crime_lon.iloc[:,3:27].sum(axis=1)
crimesum = crime_lon['Sum']
crime_lon.drop(labels=['Sum'], axis=1,inplace = True)
crime_lon.insert(3, 'Sum', crimesum)
crime_lon.head()

,MajorText,MinorText,LookUp_BoroughName,Sum,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005
0,Arson and Criminal Damage,Arson,Barking and Dagenham,127,12,6,5,3,8,5,1,5,2,5,5,11,3,5,3,6,9,8,6,4,5,6,2,2
1,Arson and Criminal Damage,Criminal Damage,Barking and Dagenham,2691,123,127,101,107,132,105,88,97,127,138,130,140,113,134,118,109,109,97,121,97,103,107,80,88
2,Burglary,Burglary - Business and Community,Barking and Dagenham,700,33,30,18,33,32,39,33,45,24,29,27,21,27,31,35,37,30,30,25,31,17,28,29,16
3,Burglary,Burglary - Residential,Barking and Dagenham,2337,77,94,84,99,94,106,164,114,107,99,96,114,96,71,67,80,97,114,130,116,123,97,57,41
4,Drug Offences,Drug Trafficking,Barking and Dagenham,177,6,8,7,10,7,7,4,5,2,6,5,9,6,11,7,7,10,12,3,13,4,6,11,11


In [43]:
# Delete unnecessary columns
crime_lon.drop(crime_lon.columns[0:2], axis=1, inplace=True)
crime_lon.drop(crime_lon.columns[2:26], axis=1, inplace=True)
crime_lon.head(10)

,LookUp_BoroughName,Sum
0,Barking and Dagenham,127
1,Barking and Dagenham,2691
2,Barking and Dagenham,700
3,Barking and Dagenham,2337
4,Barking and Dagenham,177
5,Barking and Dagenham,2194
6,Barking and Dagenham,1
7,Barking and Dagenham,1
8,Barking and Dagenham,26
9,Barking and Dagenham,1


In [44]:
# Calculate sum of incidents in the last 24 months by boroughs
crime_lon = crime_lon.groupby(['LookUp_BoroughName'], as_index=False).sum()
crime_lon.head(10)

,LookUp_BoroughName,Sum
0,Barking and Dagenham,38713
1,Barnet,59484
2,Bexley,33832
3,Brent,60193
4,Bromley,47930
5,Camden,73692
6,Croydon,65825
7,Ealing,60481
8,Enfield,58667
9,Greenwich,54865


In [45]:
# Change the sum into monthly average
crime_lon['Sum'] = crime_lon['Sum']/12
crime_lon.rename(columns={crime_lon.columns[1]:'MonthlyAverageCrimes'}, inplace=True)
crime_lon.head()

,LookUp_BoroughName,MonthlyAverageCrimes
0,Barking and Dagenham,3226.083333
1,Barnet,4957.000000
2,Bexley,2819.333333
3,Brent,5016.083333
4,Bromley,3994.166667


Getting borough with most number of Crime related incidents

In [47]:
# Getting borough with most number of Crime related incidents
crime_lon.sort_values(by='MonthlyAverageCrimes', ascending=False).head()

,LookUp_BoroughName,MonthlyAverageCrimes
32,Westminster,12018.333333
27,Southwark,6177.750000
5,Camden,6141.000000
24,Newham,5892.166667
20,Lambeth,5771.500000


Getting borough with Least number of Crime related incidents

In [49]:
# Getting borough with least number of Crime related incidents
crime_lon.sort_values(by='MonthlyAverageCrimes', ascending=True).head()

,LookUp_BoroughName,MonthlyAverageCrimes
22,London Heathrow and London City Airports,539.916667
19,Kingston upon Thames,2109.250000
26,Richmond upon Thames,2117.500000
28,Sutton,2235.500000
23,Merton,2342.916667


### 2.2 Getting list of Boroughs in London

In [ ]:
# Obtaining data from Wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_London_boroughs').text
soup = BeautifulSoup(source, 'lxml')
soup.encode("utf-8-sig")

In [138]:
London_wiki = requests.get('https://en.wikipedia.org/wiki/List_of_London_boroughs').text
print("Import successful!")

Import successful!


In [149]:
soup = BeautifulSoup(London_wiki, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})
print(table.tr.text)


Borough

Inner

Status

Local authority

Political control

Headquarters

Area (sq mi)

Population (2013 est)[1]

Co-ordinates

 Nr. in map 



In [150]:
column_names=['Borough','Inner','Status','Local authority','Political control','Headquarters','Area (sq mi)','Population','Co-ordinates','Nr. in map']
df_lon = pd.DataFrame(columns=column_names)

In [152]:
# extracting information from the table
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==10:
        df_lon.loc[len(df_lon)] = row_data
df_lon.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population,Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
1,Barnet,,,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20


In [153]:
#df_lon.drop(['Inner','Status','Local authority','Political control','Headquarters','Area (sq mi)','Nr. in map'], axis=1)
df_lon.drop(df_lon.columns[1:7], axis='columns', inplace=True)
df_lon.drop(['Nr. in map'] , axis='columns', inplace=True)
df_lon.head()

,Borough,Population,Co-ordinates
0,Barking and Dagenham [note 1],"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...
1,Barnet,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...
2,Bexley,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...
3,Brent,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...
4,Bromley,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...


In [154]:
# Removing all the extra texts in the Borough names.For ex: [note 1]
df_lon['Borough'] = df_lon['Borough'].map(lambda x: x.rstrip(']'))
df_lon['Borough'] = df_lon['Borough'].map(lambda x: x.rstrip('1234567890.'))
df_lon['Borough'] = df_lon['Borough'].str.replace('note','')
df_lon['Borough'] = df_lon['Borough'].map(lambda x: x.rstrip(' ['))
df_lon.head(10)

,Borough,Population,Co-ordinates
0,Barking and Dagenham,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...
1,Barnet,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...
2,Bexley,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...
3,Brent,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...
4,Bromley,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...
5,Camden,"229,719",51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...
6,Croydon,"372,752",51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...
7,Ealing,"342,494",51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...
8,Enfield,"320,524",51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...
9,Greenwich,"264,008",51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...


In [155]:
# Splitting the coordinates in to Latitude, Longitude
df_lon[['Coord1','Coord2','Coord3']] = df_lon['Co-ordinates'].str.split('/',expand=True)
df_lon.head()

,Borough,Population,Co-ordinates,Coord1,Coord2,Coord3
0,Barking and Dagenham,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,51°33′39″N 0°09′21″E﻿,﻿51.5607°N 0.1557°E﻿,51.5607; 0.1557﻿ (Barking and Dagenham)
1,Barnet,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,51°37′31″N 0°09′06″W﻿,﻿51.6252°N 0.1517°W﻿,51.6252; -0.1517﻿ (Barnet)
2,Bexley,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,51°27′18″N 0°09′02″E﻿,﻿51.4549°N 0.1505°E﻿,51.4549; 0.1505﻿ (Bexley)
3,Brent,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,51°33′32″N 0°16′54″W﻿,﻿51.5588°N 0.2817°W﻿,51.5588; -0.2817﻿ (Brent)
4,Bromley,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,51°24′14″N 0°01′11″E﻿,﻿51.4039°N 0.0198°E﻿,51.4039; 0.0198﻿ (Bromley)


In [158]:
# Dropping the Coordinates , Coord1, coord2, Derive Latitude and Longitude from Coord3
df_lon.drop(labels=['Co-ordinates','Coord1','Coord2'], axis=1,inplace = True)
df_lon[['Latitude','Longitude']] = df_lon['Coord3'].str.split(';',expand=True)
df_lon.head()

,Borough,Population,Coord3,Latitude,Longitude
0,Barking and Dagenham,"194,352",51.5607; 0.1557﻿ (Barking and Dagenham),51.5607,0.1557﻿ (Barking and Dagenham)
1,Barnet,"369,088",51.6252; -0.1517﻿ (Barnet),51.6252,-0.1517﻿ (Barnet)
2,Bexley,"236,687",51.4549; 0.1505﻿ (Bexley),51.4549,0.1505﻿ (Bexley)
3,Brent,"317,264",51.5588; -0.2817﻿ (Brent),51.5588,-0.2817﻿ (Brent)
4,Bromley,"317,899",51.4039; 0.0198﻿ (Bromley),51.4039,0.0198﻿ (Bromley)


In [162]:
# Remove the unwated text from Longitude
import string, re

# Droping Coord3 Column
df_lon.drop(labels=['Coord3'], axis=1,inplace = True)

# Removing all special characters
df_lon['Latitude'] = df_lon['Latitude'].apply(lambda x: x.encode('ascii', 'ignore').decode().strip())
df_lon['Longitude'] = df_lon['Longitude'].apply(lambda x: x.encode('ascii', 'ignore').decode().strip())

# Removing all the text, spaces , () from Latitude adn Longitude Column
df_lon['Latitude'] = df_lon['Latitude'].str.translate(str.maketrans('','','abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ')).replace("()", "")
df_lon['Longitude'] = df_lon['Longitude'].str.translate(str.maketrans('','','abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ ')).replace("()", "")

df_lon.head()

,Borough,Population,Latitude,Longitude
0,Barking and Dagenham,"194,352",51.5607,0.1557()
1,Barnet,"369,088",51.6252,-0.1517()
2,Bexley,"236,687",51.4549,0.1505()
3,Brent,"317,264",51.5588,-0.2817()
4,Bromley,"317,899",51.4039,0.0198()


In [163]:
# Get Unique Boroghs
df_lon['Borough'].unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
       'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney',
       'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering',
       'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea',
       'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham',
       'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
       'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster'],
      dtype=object)

### 2.3 Foursquare APIs

Using Foursquare APIs, geographicak data for London city is obtained. This data will be used to explore the venues in each of the neighbourhoods of London.

These venues will provide the different categories needed for the analysis in next step of project(Part 2). Based on the amenities present near each venue, we can determine the good neighborhoods for living in London.

In the next course of the project, this Dataset shall be merged with London crime dataset to evaluate good neighborhood with top facilites.
